In [1]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix

%matplotlib inline

In [2]:
gpu_available = tf.config.list_physical_devices('GPU')
gpu_available
# tf.debugging.set_log_device_placement(True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
loc = "data/Part 8 - Deep Learning/Section 39 - Artificial Neural Networks (ANN)/Python/"

dataset = pd.read_csv(loc + 'Churn_Modelling.csv')
print(dataset.shape)
dataset.head()

(10000, 14)


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
X = dataset.iloc[:, 3:12].values
y = dataset.iloc[:, 13].values

In [5]:
X

array([[619, 'France', 'Female', ..., 1, 1, 1],
       [608, 'Spain', 'Female', ..., 1, 0, 1],
       [502, 'France', 'Female', ..., 3, 1, 0],
       ...,
       [709, 'France', 'Female', ..., 1, 0, 1],
       [772, 'Germany', 'Male', ..., 2, 1, 0],
       [792, 'France', 'Female', ..., 1, 1, 0]], dtype=object)

In [6]:
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
ct = ColumnTransformer([("Geography, Gender", OneHotEncoder(categories='auto'),[1, 2])], remainder="passthrough") # The last arg ([0]) is the list of columns you want to transform in this step
X = ct.fit_transform(X)  
X = X[:, 1:]
print(X.shape)

(10000, 11)


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=0)

In [8]:
# Feature scaling
sc_X = StandardScaler(with_mean=False)
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [9]:
print(X_train.shape)
print(y_train.shape)

(8000, 11)
(8000,)


In [10]:
print(X_test.shape)
print(y_test.shape)

(2000, 11)
(2000,)


In [11]:
# Initialising the ANN
classifier = Sequential()

In [12]:
# Adding the inpput layer and the first hidden layer
classifier.add(keras.Input(shape=(11,)))
classifier.add(Dense(units=6, kernel_initializer="glorot_uniform", activation='relu'))
classifier.add(Dense(units=6, kernel_initializer="glorot_uniform", activation='relu'))
classifier.add(Dense(units=1, kernel_initializer="glorot_uniform", activation='sigmoid'))

2024-08-10 11:57:09.310428: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Max
2024-08-10 11:57:09.310448: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2024-08-10 11:57:09.310452: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2024-08-10 11:57:09.310469: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-08-10 11:57:09.310481: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [13]:
# Compiling th ANN
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
classifier.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 6)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │            42 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             7 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 121 (484.00 B)

 Trainable params: 121 (484.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Fitting the ANN to the Trainning set
classifier.fit(X_train, y_train, batch_size=10, epochs=100)

Epoch 1/100


2024-08-10 11:57:09.624809: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


800/800 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.7849 - loss: 0.5652
Epoch 2/100
800/800 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7988 - loss: 0.4726
Epoch 3/100
800/800 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8088 - loss: 0.4440
Epoch 4/100
800/800 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.8199 - loss: 0.4339
Epoch 5/100
800/800 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8175 - loss: 0.4263
Epoch 6/100
800/800 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8180 - loss: 0.4331
Epoch 7/100
800/800 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8186 - loss: 0.4227
Epoch 8/100
800/800 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8348 - loss: 0.4096
Epoch 9/100
800/800 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8249 - loss: 0.4214
Epoch 10/100
800/800 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8126 - loss: 0.4303
Epoch 11/100
800/800 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8254 - loss: 0.4231
Epoch 12/100
800/800 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step

In [ ]:
# model_file = 'models/ann_classifier.keras'
# try:
#   # Specify an invalid GPU device
#   with tf.device('/GPU:0'):
#     early_stop = tf.keras.callbacks.EarlyStopping(monitor='accuracy', 
#                     patience = 3, restore_best_weights = False)
#     checkpoint = tf.keras.callbacks.ModelCheckpoint(model_file, 
#                         monitor="loss", mode="min", 
#                         save_best_only=True, verbose=1)
#     history = classifier.fit(X_train, y_train, batch_size=10, epochs=100, callbacks=[early_stop, checkpoint])
# except RuntimeError as e:
#   print(e)
#loading model weights
# classifier.load_weights(model_file)

In [ ]:
# try:
#   # Specify an invalid GPU device
#   with tf.device('/GPU:0'):
#     history = classifier.fit(X_train, y_train, batch_size=10, epochs=100)
# except RuntimeError as e:
#   print(e)

In [ ]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > .5)
y_pred

In [ ]:
cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
# array([[1533,   62],
#        [ 255,  150]])